The questions I am trying to answer:
1) What was the frequency of trade or how many trades per day
2) Average profit and loss from each trade
3) Whats the risk reward ratio
4) Average holding period per security

In [1]:
import pandas as pd
import numpy as np

In [2]:
with open('All for tony, thinkorswim trade challenge winner2.txt') as f:
    table = pd.read_table(f, sep='\s+', engine='python', header=None,   lineterminator='\n')
# sep=' |:', engine='python'

In [3]:
table.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,11/04/2016,02:21:43,PM,BOT,"+5,500",NEM,100,4,NOV,16,37.5,CALL,@.15,"$82,500.00"
1,11/04/2016,01:57:41,PM,SOLD,-300,UVXY,100,11,NOV,16,18.5,PUT,@2.25,"-$67,500.00"
2,11/04/2016,01:35:00,PM,BOT,+666,VXX,100,11,NOV,16,36.0,CALL,@3.40,"$226,440.00"
3,11/04/2016,01:27:59,PM,BOT,+200,TSLA,100,11,NOV,16,190.0,PUT,@4.60,"$92,000.00"
4,11/04/2016,01:07:58,PM,BOT,"+1,200",GPRO,100,11,NOV,16,11.5,PUT,@.70,"$84,000.00"


In [4]:
# table["1"]= table["2"] + table["3"]
table.iloc[:,0]=table.iloc[:,0]+" "+table.iloc[:,1]+""+table.iloc[:,2]
# table=table.drop(table.columns[[1,2]],axis=1)
table.iloc[:,5]=table.iloc[:,5]+" "+table.iloc[:,6].map(str)+" "+table.iloc[:,7].map(str)+" "+table.iloc[:,8].map(str)+" "+table.iloc[:,9].map(str)+" "+ \
table.iloc[:,10].map(str)+" "+table.iloc[:,11].map(str)
table=table.drop(table.columns[[1,2,6,7,8,9,10,11]],axis=1)
table.head()

,0,3,4,5,12,13
0,11/04/2016 02:21:43PM,BOT,"+5,500",NEM 100 4 NOV 16 37.5 CALL,@.15,"$82,500.00"
1,11/04/2016 01:57:41PM,SOLD,-300,UVXY 100 11 NOV 16 18.5 PUT,@2.25,"-$67,500.00"
2,11/04/2016 01:35:00PM,BOT,+666,VXX 100 11 NOV 16 36.0 CALL,@3.40,"$226,440.00"
3,11/04/2016 01:27:59PM,BOT,+200,TSLA 100 11 NOV 16 190.0 PUT,@4.60,"$92,000.00"
4,11/04/2016 01:07:58PM,BOT,"+1,200",GPRO 100 11 NOV 16 11.5 PUT,@.70,"$84,000.00"


In [5]:
import re
# table.iloc[:,13]=table.iloc[:,13].replace("@", "")
table.columns=['Time','Position','Quantity','Security','Price','Cash_flow']
# table["Price"].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
# table["Quantity"].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
# table["cash_flow"].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
# table.cash_flow = table.cash_flow.str.translate(None, "$") #use if u downgrading to python 2
# table.cash_flow = str(re.sub('[$]', '', str(table.cash_flow)))
table.Price =table.Price.str.replace('@','')
table.Price =table.Price.astype(float)
table.Cash_flow =table.Cash_flow.str.replace('$','')
table.Cash_flow =table.Cash_flow.str.replace(',','')
table.Cash_flow =table.Cash_flow.astype(float)
table.Quantity =table.Quantity.str.replace(',','')
table.Quantity = table.Quantity.astype(float)
table.head()

,Time,Position,Quantity,Security,Price,Cash_flow
0,11/04/2016 02:21:43PM,BOT,5500,NEM 100 4 NOV 16 37.5 CALL,0.15,82500
1,11/04/2016 01:57:41PM,SOLD,-300,UVXY 100 11 NOV 16 18.5 PUT,2.25,-67500
2,11/04/2016 01:35:00PM,BOT,666,VXX 100 11 NOV 16 36.0 CALL,3.40,226440
3,11/04/2016 01:27:59PM,BOT,200,TSLA 100 11 NOV 16 190.0 PUT,4.60,92000
4,11/04/2016 01:07:58PM,BOT,1200,GPRO 100 11 NOV 16 11.5 PUT,0.70,84000


In [6]:
# table.groupby('security').count()
# table.security.value_counts()

In [7]:
import datetime
import locale
locale.setlocale(locale.LC_ALL, "") # for datetime.strptime

# datetime_object = datetime.strptime('11/04/2016 02:21:43PM', '%x %I:%M:%S%p')
table.Time = pd.to_datetime(table.Time,format="%x %I:%M:%S%p")

table.head()

,Time,Position,Quantity,Security,Price,Cash_flow
0,2016-11-04 14:21:43,BOT,5500,NEM 100 4 NOV 16 37.5 CALL,0.15,82500
1,2016-11-04 13:57:41,SOLD,-300,UVXY 100 11 NOV 16 18.5 PUT,2.25,-67500
2,2016-11-04 13:35:00,BOT,666,VXX 100 11 NOV 16 36.0 CALL,3.40,226440
3,2016-11-04 13:27:59,BOT,200,TSLA 100 11 NOV 16 190.0 PUT,4.60,92000
4,2016-11-04 13:07:58,BOT,1200,GPRO 100 11 NOV 16 11.5 PUT,0.70,84000


In [8]:
# table.security.unique()
security = table.Security.unique()
average_holding_period =[0]* len(security)
total_profit =[0]* len(security)
average_profit =[0]* len(security)
number_of_trades=[0]* len(security)

trade_analyzis = {'security': security,'average_holding_period':[0]* len(security),\
                  'total_profit':[0]* len(security),'average_profit':[0]* len(security),'number_of_trades':[0]* len(security),\
                  'outstanding_quantity':[0]* len(security),'start_time':[0]* len(security)}


trade_analyzis = pd.DataFrame(trade_analyzis)
trade_analyzis.head()

,average_holding_period,average_profit,number_of_trades,outstanding_quantity,security,start_time,total_profit
0,0,0,0,0,NEM 100 4 NOV 16 37.5 CALL,0,0
1,0,0,0,0,UVXY 100 11 NOV 16 18.5 PUT,0,0
2,0,0,0,0,VXX 100 11 NOV 16 36.0 CALL,0,0
3,0,0,0,0,TSLA 100 11 NOV 16 190.0 PUT,0,0
4,0,0,0,0,GPRO 100 11 NOV 16 11.5 PUT,0,0


Security | Average Holding Period | Total Profit | Average Profit | Number of trades | 


In [19]:
table.sort_values(by='Time',inplace=True)
table.head()

# z.index[0] + dt.timedelta(minutes=5)


,Time,Position,Quantity,Security,Price,Cash_flow
117,2016-10-11 10:10:19,BOT,200,VXX 100 28 OCT 16 34.5 CALL,1.50,30000
116,2016-10-11 13:50:28,SOLD,-200,VXX 100 28 OCT 16 34.5 CALL,1.86,-37200
115,2016-10-11 14:11:19,BOT,300,VXX 100 21 OCT 16 34.5 PUT,1.48,44400
114,2016-10-11 14:22:10,BOT,100,SPY 100 14 OCT 16 213.0 CALL,1.40,14000
113,2016-10-11 14:27:21,SOLD,-300,VXX 100 21 OCT 16 34.5 PUT,1.60,-48000


In [20]:
for indx,row in table.iterrows():
    if trade_analyzis[trade_analyzis.security==row.Security].outstanding_quantity.item() == 0:
        trade_analyzis.loc[trade_analyzis.security==row.Security,'number_of_trades']+= 1
#             trade_analyzis[trade_analyzis.security==row.Security].number_of_trades+1
            
    trade_analyzis.loc[trade_analyzis.security==row.Security,'total_profit'] = row.Cash_flow
#     print((trade_analyzis[trade_analyzis.security==row.Security].outstanding_quantity)+ int(row.Quantity))
    trade_analyzis.loc[trade_analyzis.security==row.Security,'outstanding_quantity'] += int(row.Quantity)
#     trade_analyzis.loc[trade_analyzis.security==row.Security,'outstanding_quantity']+int(row.Quantity)
    print(trade_analyzis[trade_analyzis.security==row.Security].outstanding_quantity)
    
    if (trade_analyzis[trade_analyzis.security==row.Security].outstanding_quantity.item() != 0):
        trade_analyzis.loc[trade_analyzis.security==row.Security,'start_time'] = row.Time
    else:
        print("poda",trade_analyzis[trade_analyzis.security==row.Security].start_time.item())
        print("maire",row.Time )
        trade_analyzis.loc[trade_analyzis.security==row.Security,'average_holding_period']+=\
        ((row.Time- trade_analyzis.loc[trade_analyzis.security==row.Security,'start_time'])/datetime.timedelta(hours=1))
        trade_analyzis[trade_analyzis.security==row.Security].average_holding_period = 0


    

54    200
Name: outstanding_quantity, dtype: int64
54    0
Name: outstanding_quantity, dtype: int64
poda 2016-10-11 10:10:19
maire 2016-10-11 13:50:28
36    300
Name: outstanding_quantity, dtype: int64
53    100
Name: outstanding_quantity, dtype: int64
36    0
Name: outstanding_quantity, dtype: int64
poda 2016-10-11 14:11:19
maire 2016-10-11 14:27:21


C:\Users\saeed\Anaconda3\lib\site-packages\pandas\core\generic.py:2387: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


52    10
Name: outstanding_quantity, dtype: int64
52    400
Name: outstanding_quantity, dtype: int64
53    0
Name: outstanding_quantity, dtype: int64
poda 2016-10-11 14:22:10
maire 2016-10-11 14:43:11
52    0
Name: outstanding_quantity, dtype: int64
poda 2016-10-11 14:31:19
maire 2016-10-11 14:54:37
49    1000
Name: outstanding_quantity, dtype: int64
51    1100
Name: outstanding_quantity, dtype: int64
51    0
Name: outstanding_quantity, dtype: int64
poda 2016-10-12 12:48:58
maire 2016-10-12 13:00:17
50    1000
Name: outstanding_quantity, dtype: int64
50    0
Name: outstanding_quantity, dtype: int64
poda 2016-10-12 13:05:11
maire 2016-10-12 13:10:01
48    630
Name: outstanding_quantity, dtype: int64
48   -630
Name: outstanding_quantity, dtype: int64
49    0
Name: outstanding_quantity, dtype: int64
poda 2016-10-12 12:46:13
maire 2016-10-12 14:48:50
48    0
Name: outstanding_quantity, dtype: int64
poda 2016-10-12 14:47:57
maire 2016-10-12 14:57:23
47    500
Name: outstanding_quantity, dty

In [21]:
trade_analyzis
trade_analyzis.total_profit.sum()
trade_analyzis

,average_holding_period,average_profit,number_of_trades,outstanding_quantity,security,start_time,total_profit
0,0.000000,0,1,5500,NEM 100 4 NOV 16 37.5 CALL,2016-11-04 14:21:43,82500
1,1.099167,0,1,0,UVXY 100 11 NOV 16 18.5 PUT,2016-11-04 12:51:44,-67500
2,0.000000,0,1,276,VXX 100 11 NOV 16 36.0 CALL,2016-11-04 13:35:00,226440
3,0.123056,0,2,200,TSLA 100 11 NOV 16 190.0 PUT,2016-11-04 13:27:59,92000
4,0.000000,0,1,1200,GPRO 100 11 NOV 16 11.5 PUT,2016-11-04 13:07:58,84000
5,27.163056,0,1,0,GDX 100 11 NOV 16 25.0 CALL,2016-11-03 08:36:15,-79000
6,51.593056,0,1,0,UVXY 100 11 NOV 16 18.5 CALL,2016-11-01 11:11:31,-121800
7,51.215556,0,1,0,YHOO 100 11 NOV 16 41.0 PUT,2016-11-01 11:33:19,-81200
8,6.115556,0,1,0,NEM 100 11 NOV 16 37.5 CALL,2016-11-03 08:35:04,-81900
9,3.255556,0,1,0,AET 100 4 NOV 16 105.0 PUT,2016-11-03 11:20:33,-9600


In [14]:
# trade_analyzis.loc[trade_analyzis.loc["VXX 100 21 OCT 16 34.5 PUT"].,average_holding_period]
trade_analyzis.loc[trade_analyzis.security=="VXX 100 14 OCT 16 34.5 PUT",'security']
# df.loc[df['column_name'] == some_value]
# trade_analyzis.[trade_analyzis==row.Security].average_holding_period = 0


52    VXX 100 14 OCT 16 34.5 PUT
Name: security, dtype: object

In [15]:
row=table.iloc[1,:]
(row.Time- trade_analyzis[trade_analyzis.security==row.Security].start_time)
# row.Security

ValueError: Cannot add integral value to Timestamp without offset.

In [13]:
(row.Time- trade_analyzis.loc[trade_analyzis.security==row.Security,'start_time'])/datetime.timedelta(hours=1)

ValueError: Cannot add integral value to Timestamp without offset.